In [13]:
import cv2
import csv
import math
import numpy as np
import os
from datetime import datetime
from pathlib import Path
import mediapipe as mp

# Инициализация модели MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

CSV_FILENAME = "анализ_датасета.csv"
SUPPORTED_EXT = ('.jpg', '.jpeg', '.png', '.webp')

LABELS = [
    "шизоидный", "параноидальный", "нарциссический", "психопатический",
    "компульсивный", "истерический", "депрессивный", "мазохистический"
]

def calculate_distance(p1, p2, img_shape):
    h, w = img_shape[:2]
    return math.hypot((p1.x - p2.x) * w, (p1.y - p2.y) * h)

def process_image(img_path):
    result = {
        'имя_файла': img_path.name,
        'дата_обработки': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'форма_рта': None,
        'форма_губ': None,
        'форма_носа': None,
        'экстраверсия_или_интроверсия': None,
        'ошибка': None,
        'метка': None,
        'тип_мышления': None,
        'путь_папки': None
    }

    try:
        img = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError("Не удалось прочитать файл")

        results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            raise ValueError("Лицо не обнаружено")

        landmarks = results.multi_face_landmarks[0].landmark
        h, w = img.shape[:2]

        # Замеры:
        mouth_width = calculate_distance(landmarks[61], landmarks[291], (h, w))
        lip_height = calculate_distance(landmarks[13], landmarks[14], (h, w))
        nose_length = calculate_distance(landmarks[1], landmarks[2], (h, w))
        nose_width = calculate_distance(landmarks[98], landmarks[327], (h, w))

        # Описание форм
        lip_description = "Полные губы" if lip_height / mouth_width > 0.18 else "Тонкие губы"
        mouth_description = "Большой рот" if mouth_width > 60 else "Маленький рот"
        nose_description = "Нос длинный, широкий" if nose_length > 50 and nose_width > 40 else "Нос короткий, маленький"

        # Классификация
        features = []
        if "Полные" in lip_description:
            features.append('Экстраверсия')
        else:
            features.append('Интроверсия')

        if "Большой" in mouth_description:
            features.append('Экстраверсия')
        else:
            features.append('Интроверсия')

        if "длинный" in nose_description:
            features.append('Экстраверсия')
        else:
            features.append('Интроверсия')

        extravert_score = features.count('Экстраверсия')
        introvert_score = features.count('Интроверсия')

        if extravert_score >= 2:
            personality = 'Экстраверт'
        else:
            personality = 'Интроверт'

        # Сохраняем описания
        result.update({
            'форма_рта': mouth_description,
            'форма_губ': lip_description,
            'форма_носа': nose_description,
            'экстраверсия_или_интроверсия': personality
        })

    except Exception as e:
        result['ошибка'] = str(e)

    return result

def save_results(data, filename):
    file_exists = os.path.exists(filename)
    with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def process_dataset(root_path):
    root = Path(root_path)
    if not root.exists():
        print(f"Ошибка: Папка {root} не найдена")
        return

    e_folder = next((f for f in root.iterdir() if f.is_dir() and f.name.startswith('E (')), None)
    i_folder = next((f for f in root.iterdir() if f.is_dir() and f.name.startswith('I (')), None)

    for top_folder in [e_folder, i_folder]:
        if top_folder is None:
            print("Не найдена одна из главных папок.")
            continue

        thinking_type = top_folder.name.split('(', 1)[1].rstrip(')')

        folders = sorted([f for f in top_folder.iterdir() if f.is_dir()])

        for idx, folder in enumerate(folders):
            label = LABELS[idx % len(LABELS)]
            print(f"\n▶ Обработка папки {top_folder.name}/{folder.name} (тип: {label})")

            image_files = []
            for ext in SUPPORTED_EXT:
                image_files.extend(folder.glob(f'*{ext}'))
                image_files.extend(folder.glob(f'*{ext.upper()}'))

            for img_path in image_files:
                print(f" → {img_path.name}")
                result = process_image(img_path)
                result['метка'] = label
                result['тип_мышления'] = thinking_type
                result['путь_папки'] = f"{top_folder.name}/{folder.name}"
                save_results(result, CSV_FILENAME)

    print(f"\n✅ Все данные сохранены в {CSV_FILENAME}")

if __name__ == "__main__":
    dataset_root = input("Введите путь к папке data (например: data): ").strip()
    process_dataset(dataset_root)



▶ Обработка папки E (Экстраверсия)/1 Дк (тип: шизоидный)
 → -_OnqZ0eJas.jpg
 → 1200а.jpg
 → 16.jpg
 → 16028.jpg
 → 16503а (1).jpg
 → 16503а (2).jpg
 → 18474.jpg
 → 1mvbji9E8j8.jpg
 → 20729.jpg
 → 21.jpg
 → 2510.jpg
 → 2656b.jpg
 → 2gKSVLBc23E.jpg
 → 2Si3ZA8zLRo.jpg
 → 31447а (1).jpg
 → 31447а (2).jpg
 → 42892.jpg
 → 43BhsyTyaTk.jpg
 → 4Fybt5QsBnM.jpg
 → 4vg79ZoHb-4.jpg
 → 5199.jpg
 → 67CHcbI07YE.jpg
 → 6Qxws_0mXf4.jpg
 → 70301.jpg
 → 8201а (1).jpg
 → 8201а (2).jpg
 → 9r7B-zSbmuE.jpg
 → a5H7iysrPGg.jpg
 → B9nEbJGDY6M.jpg
 → brittney griner basketball1.jpg
 → brittney griner basketball1a.jpg
 → brittney griner basketball1s.jpg
 → brittney griner basketball2.jpg
 → brittney griner basketball3.jpg
 → brittney griner basketball4.jpg
 → brittney griner basketball5.jpg
 → brittney griner basketball6.jpg
 → brittney griner basketball6c.jpg
 → brittney griner basketball76.jpg
 → brittney griner basketball7ф.jpg
 → brittney griner basketball7ы.jpg
 → brittney griner basketball8.jpg
 → brittney_